In [43]:
import pandas as pd
import re
import json
from collections import defaultdict
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import xlsxwriter
import numpy as np

warnings.filterwarnings("ignore", category=FutureWarning)

In [44]:
address_similarity_threshold = 75

classification_path = "content/CustomerClusterReport.xlsx"
site_path = "content/MILGARD DATA 26-07-25 DEV2 (1).xlsx"
site_sheet_name = "Site and Site Use"
output_path = "content/FinalReport.xlsx"
address_cluster_path = "content/AddressClusterReport.xlsx"

classification_df = pd.read_excel(classification_path)
df_sites = pd.read_excel(site_path, site_sheet_name)

report_cols = [
    'Site ID', 'Site Name', 'Site Purpose',
    'Address Line 1', 'Address Line 2', 'Mail Stop', 'City', 'State/Province',
    'Postal Code', 'County', 'Country', 'Party ID'
]

address_cols = [
    'Address Line 1', 'Address Line 2', 'Mail Stop', 'City',
    'State/Province', 'Postal Code', 'County', 'Country'
]

columns_to_keep = [
    'classification', 'Party Name', 'Party ID',
    'Site ID', 'Address Line 1', 'Address Line 2', 'City',
    'State/Province', 'Postal Code', 'County', 'Country'
]

In [45]:
df_sites = df_sites[report_cols]
merged_df = classification_df.merge(df_sites, on='Party ID', how='inner').fillna("")

def normalize_text(s):
    return re.sub(r"\s+", " ", re.sub(r"[^\w\s]", " ", str(s).lower())).strip()

def build_weighted_text(row):
    parts = []
    for col in address_cols:
        val = str(row.get(col, '')).strip()
        if col == "Postal Code" and "-" in val:
            val = val.split("-")[0].strip()
        parts.append((val + " ") * 3 if col in ["Address Line 1", "Postal Code"] else val)
    return normalize_text(" ".join(parts))

merged_df["full_address"] = merged_df.apply(build_weighted_text, axis=1)

In [46]:
sbert = SentenceTransformer('all-MiniLM-L6-v2')
embs = sbert.encode(merged_df["full_address"].tolist(), convert_to_tensor=False, normalize_embeddings=True)

merged_df["row_id"] = merged_df.index
party_site_map = defaultdict(list)
for idx, row in merged_df.iterrows():
    party_site_map[row["Party ID"]].append(row["row_id"])

similarity_column = []
for idx1, row1 in merged_df.iterrows():
    pid1, emb1 = row1["Party ID"], embs[row1["row_id"]].reshape(1, -1)
    sims = []
    for pid2, indices in party_site_map.items():
        if pid1 == pid2:
            continue
        for idx2 in indices:
            row2 = merged_df.loc[idx2]
            emb2 = embs[idx2].reshape(1, -1)
            sim = float(cosine_similarity(emb1, emb2)[0][0])
            sims.append({"To Party": int(pid2), "To Site": str(row2["Site ID"]), "Similarity": f"{sim * 100:.2f}%"})
    similarity_column.append(json.dumps(sims, ensure_ascii=False))

merged_df["Similarity to Other Parties"] = similarity_column
merged_df["Similarity Parsed"] = merged_df["Similarity to Other Parties"].apply(json.loads)

merged_df.to_excel("content/merged_Cluster.xlsx", index=False)

In [47]:
final_clusters = []

for class_id, group in merged_df.groupby("classification"):
    pids_in_class = set(group["Party ID"])

    qualified_rows = []
    for _, row in group.iterrows():
        sims = row["Similarity Parsed"]
        for entry in sims:
            if float(entry["Similarity"].replace('%', '')) > address_similarity_threshold and entry["To Party"] in pids_in_class:
                qualified_rows.append(row)
                break

    if len(qualified_rows) >= 2:
        final_clusters.append(pd.DataFrame(qualified_rows))

if final_clusters:
    result_df = pd.concat(final_clusters).copy()
    result_df.sort_values(by=["classification", "Party ID"], inplace=True)
    old_to_new_class = {old: new for new, old in enumerate(sorted(result_df["classification"].unique()), start=1)}
    result_df["classification"] = result_df["classification"].map(old_to_new_class)
else:
    result_df = pd.DataFrame(columns=merged_df.columns)

type_labels_map = {}

for class_id, group in result_df.groupby("classification"):
    is_pure_customer_duplication = True

    for sims in group["Similarity to Other Parties"]:
        parsed = json.loads(sims)
        sims_floats = [float(entry["Similarity"].replace('%', '')) for entry in parsed]

        if 100.0 not in sims_floats:
            is_pure_customer_duplication = False
            break

        if any(sim != 100.0 for sim in sims_floats):
            is_pure_customer_duplication = False
            break

    label = "Customer Duplication" if is_pure_customer_duplication else "Customer + Address Duplication"
    type_labels_map[class_id] = label

result_df["type"] = result_df["classification"].map(type_labels_map)
result_df = result_df[["type"] + columns_to_keep]

result_df.drop(columns=["Similarity Parsed", "full_address", "row_id"], errors='ignore', inplace=True)
result_df = result_df[["type"] + columns_to_keep]

addr_dup_df = pd.read_excel(address_cluster_path)
addr_dup_df["type"] = "Address Duplication"
final_df = pd.concat([result_df, addr_dup_df], ignore_index=True)

final_df = final_df.replace([np.inf, -np.inf], np.nan)
final_df = final_df.fillna("")

final_df = final_df[["type"] + columns_to_keep]

In [ ]:
site_df = pd.read_excel(site_path, sheet_name="Site and Site Use")
site_account_df = site_df[["Site ID", "Account Number"]]

# Merge Account Number into final_df using Site ID
final_df = final_df.merge(site_account_df, on="Site ID", how="left")

# Move Account Number column after Site ID (optional)
cols = final_df.columns.tolist()
if "Account Number" in cols and "Site ID" in cols:
    site_id_idx = cols.index("Site ID")
    # Remove Account Number and insert after Site ID
    cols.remove("Account Number")
    cols.insert(site_id_idx + 1, "Account Number")
    final_df = final_df[cols]

In [ ]:
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    final_df.to_excel(writer, index=False, sheet_name='Filtered')
    
    workbook = writer.book
    worksheet = writer.sheets['Filtered']

    header_format = workbook.add_format({
        'bold': True,
        'bg_color': '#D3D3D3',
        'border': 1
    })

    border_format = workbook.add_format({'border': 1})
    highlight_format = workbook.add_format({'bg_color': '#FFFACD', 'border': 1})

    for col_num, column_name in enumerate(final_df.columns):
        worksheet.write(0, col_num, column_name, header_format)

    worksheet.autofilter(0, 0, 0, len(final_df.columns) - 1)

    max_row = final_df.shape[0]
    max_col = final_df.shape[1]

    address_col_indices = [final_df.columns.get_loc(col) for col in address_cols if col in final_df.columns]

    for class_id, group in final_df.groupby('classification'):
        cluster_indices = group.index.tolist()
        for col_idx in address_col_indices:
            col_name = final_df.columns[col_idx]
            mode_val = group[col_name].mode().iloc[0] if not group[col_name].mode().empty else None
            highlight_entire_col = any(
                mode_val is not None and cell_val != mode_val
                for cell_val in group[col_name]
            )
            for row_idx in cluster_indices:
                excel_row = row_idx + 1
                cell_val = final_df.iloc[row_idx, col_idx]
                if highlight_entire_col:
                    worksheet.write(excel_row, col_idx, cell_val, highlight_format)
                else:
                    worksheet.write(excel_row, col_idx, cell_val, border_format)
        for row_idx in cluster_indices:
            for col in range(max_col):
                if col not in address_col_indices:
                    excel_row = row_idx + 1
                    worksheet.write(excel_row, col, final_df.iloc[row_idx, col], border_format)

    for idx, col in enumerate(final_df.columns):
        col_data = final_df[col].astype(str)
        max_len = max([len(str(col))] + col_data.map(len).tolist())
        worksheet.set_column(idx, idx, max_len + 2)

output_path

'content/FinalReport.xlsx'